<a href="https://colab.research.google.com/github/slawek-programmer/dw_matrix_road_sign/blob/master/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install hyperopt

In [16]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

import os
import datetime

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color,exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [4]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim==1: y_train = to_categorical(y_train)
if y_test.ndim==1: y_test = to_categorical(y_test)

In [0]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model,X_train,y_train,params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  logdir = os.path.join("logs",datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tenserboard_callback = tf.keras.callbacks.TensorBoard(logdir,histogram_freq=1)

  model.fit(
      X_train, 
      y_train,
      batch_size=params_fit.get('batch_size',128),
      epochs=params_fit.get('epochs',5),
      verbose=params_fit.get('verbose',1),
      validation_data=params_fit.get('validation_data',(X_train,y_train)),
      callbacks=[tenserboard_callback]
      )
  
  return model

In [0]:
def predict(model_trained,X_test,y_test,scoring=accuracy_score):

  y_test_norm = np.argmax(y_test,axis=1)
  y_pred_prob = model_trained.predict(X_test)
  y_pred=np.argmax(y_pred_prob,axis=1)
  return scoring(y_test_norm,y_pred)

In [0]:
def train_and_predict(model):
  model_trained = train_model(model,X_train,y_train)
  return predict(model_trained,X_test,y_test)

def get_cnn_v5(input_shape,num_classes):
  return Sequential([
                     Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
                     Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
                     MaxPool2D(),
                     Dropout(0.3),

                     Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
                     Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
                     MaxPool2D(),
                     Dropout(0.3),

                     Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
                     Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
                     MaxPool2D(),
                     Dropout(0.3),

                     Flatten(),

                     Dense(1024, activation='relu'),
                     Dropout(0.3),

                     Dense(1024, activation='relu'),
                     Dropout(0.3),

                     Dense(num_classes, activation='softmax'),
  ])

# train_and_predict(get_cnn_v5(input_shape,num_classes))

In [0]:
def get_model(params):
  return Sequential([
                     Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
                     Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
                     Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
                     MaxPool2D(),
                     Dropout(params['dropout_cnn_block_one']),

                     Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
                     Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),                     
                     Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
                     MaxPool2D(),
                     Dropout(params['dropout_cnn_block_two']),

                     Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
                     Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
                     Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
                     MaxPool2D(),
                     Dropout(params['dropout_cnn_block_three']),

                     Flatten(),

                     Dense(1024, activation='relu'),
                     Dropout(params['dropout_dense_block_one']),

                     Dense(1024, activation='relu'),
                     Dropout(params['dropout_dense_block_two']),

                     Dense(num_classes, activation='softmax'),
  ])

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(
      X_train, 
      y_train,
      batch_size=int(params.get('batch_size',128)),
      epochs=5,
      verbose=0
      )
  
  accuracy = model.evaluate(X_test,y_test, verbose=0)[1]
  print(params, 'accuracy={}'.format(accuracy))
  
  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [46]:
space = {
    'batch_size': hp.quniform('batch_size', 50, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0, 1),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0, 1),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0, 1),

    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0, 1),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0, 1),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 90.0, 'dropout_cnn_block_one': 0.576557461073224, 'dropout_cnn_block_three': 0.9768002357477663, 'dropout_cnn_block_two': 0.44343486936885756, 'dropout_dense_block_one': 0.025753947446772285, 'dropout_dense_block_two': 0.03635783740209575}
accuracy=0.17301587760448456
{'batch_size': 50.0, 'dropout_cnn_block_one': 0.4605257592081694, 'dropout_cnn_block_three': 0.7852197794495811, 'dropout_cnn_block_two': 0.009020199137197427, 'dropout_dense_block_one': 0.20609734455679707, 'dropout_dense_block_two': 0.9260894332104198}
accuracy=0.6755102276802063
{'batch_size': 120.0, 'dropout_cnn_block_one': 0.5319624587764431, 'dropout_cnn_block_three': 0.08596100771492698, 'dropout_cnn_block_two': 0.06743570634689322, 'dropout_dense_block_one': 0.9890483344936043, 'dropout_dense_block_two': 0.6059591233238699}
accuracy=0.2566893398761749
{'batch_size': 150.0, 'dropout_cnn_block_one': 0.8926935135138744, 'dropout_cnn_block_three': 0.8128308307271263, 'dropout_cnn_block_two': 0.466021605